<a href="https://colab.research.google.com/github/ronie38/Paraphrasing-the-sentence/blob/main/Paraphrasing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sequence to sequence task
1. BART

for 700 MB datasets we need RAM>32GB and 4 parallel GPU.

In [12]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
cd /content/drive/MyDrive/Projects/Dataset

/content/drive/MyDrive/Projects/Dataset


In [3]:
# !unzip /content/drive/MyDrive/Projects/Dataset.zip

Archive:  /content/drive/MyDrive/Projects/Dataset.zip
  inflating: Dataset/Dataset_links.txt  
  inflating: Dataset/dev_2k.tsv      
  inflating: Dataset/msr_paraphrase_test.txt  
  inflating: Dataset/msr_paraphrase_train.txt  
  inflating: Dataset/parabank.5m.meta  
  inflating: Dataset/parabank.5m.tsv  
  inflating: Dataset/questions.csv   
  inflating: Dataset/README          
  inflating: Dataset/test_2k.tsv     
  inflating: Dataset/train.tsv       


In [4]:
!pip install simpletransformers==0.60.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.5 MB/s eta

In [5]:
import warnings
import os
from datetime import datetime
import logging
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

In [6]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [7]:
def load_data(
    file_path, input_text_column, target_text_column, label_column, keep_label=1
):
    df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df

In [8]:
def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string

# Google Data

In [17]:
# Google Data
train_df = pd.read_csv("train.tsv", sep="\t", error_bad_lines=False).astype(str)
eval_df = pd.read_csv("dev.tsv", sep="\t", error_bad_lines=False).astype(str)


train_df = train_df.loc[train_df["label"] == "1"]
eval_df = eval_df.loc[eval_df["label"] == "1"]

train_df = train_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"
print(train_df)
print(eval_df)

<ipython-input-17-e266fafadd48>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train_df = pd.read_csv("train.tsv", sep="\t", error_bad_lines=False).astype(str)
Skipping line 1250: expected 4 fields, saw 5
Skipping line 1680: expected 4 fields, saw 5
Skipping line 1810: expected 4 fields, saw 5
Skipping line 2161: expected 4 fields, saw 5
Skipping line 4531: expected 4 fields, saw 5
Skipping line 13379: expected 4 fields, saw 5
Skipping line 15111: expected 4 fields, saw 5
Skipping line 15457: expected 4 fields, saw 5
Skipping line 17265: expected 4 fields, saw 5
Skipping line 17986: expected 4 fields, saw 5
Skipping line 18421: expected 4 fields, saw 5
Skipping line 29019: expected 4 fields, saw 5
Skipping line 36181: expected 4 fields, saw 5
Skipping line 41179: expected 4 fields, saw 5
Skipping line 43631: expected 4 fields, saw 5
Skipping line 45641: expected 4 fields, saw 5
Skipping lin

Empty DataFrame
Columns: [input_text, target_text, prefix]
Index: []
                                             input_text  \
1     They were there to enjoy us and they were ther...   
2     After the end of the war in June 1902 , Higgin...   
3     From the merger of the Four Rivers Council and...   
4     The group toured extensively and became famous...   
5     Kathy and her husband Pete Beale ( Peter Dean ...   
...                                                 ...   
7993  In Advent , the traditional `` Tauberbischofsh...   
7994  In 2002 , the song was released by British pro...   
7995  Tommy Connolly , who plays Rory Jennings , pla...   
7996  Monroe Meadows , in Yosemite valley near Brida...   
7998  In 2014 the site launched iOS and Android appl...   

                                            target_text      prefix  
1     They were there for us to enjoy and they were ...  paraphrase  
2     In August , after the end of the war in June 1...  paraphrase  
3     Shawne

<ipython-input-17-e266fafadd48>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  eval_df = pd.read_csv("dev.tsv", sep="\t", error_bad_lines=False).astype(str)


## MSRP dataset

In [19]:
# MSRP Data
train_df = pd.concat(
    [
        train_df,
        load_data("msr_paraphrase_train.txt", "#1 String", "#2 String", "Quality"),
    ]
)
eval_df = pd.concat(
    [
        eval_df,
        load_data("msr_paraphrase_test.txt", "#1 String", "#2 String", "Quality"),
    ]
)

print(train_df)
print(eval_df)

                                             input_text  \
0     Amrozi accused his brother, whom he called "th...   
2     They had published an advertisement on the Int...   
4     The stock rose $2.11, or about 11 percent, to ...   
5     Revenue in the first quarter of the year dropp...   
7     The DVD-CCA then appealed to the state Supreme...   
...                                                 ...   
3931  Knox County Health Department is following nat...   
3932  The new rules will allow a single company to o...   
3933  At this point, Mr. Brando announced: 'Somebody...   
3935  We have concluded that the outlook for price s...   
3936  The notification was first reported Friday by ...   

                                            target_text      prefix  
0     Referring to him as only "the witness", Amrozi...  paraphrase  
2     On June 10, the ship's owners had published an...  paraphrase  
4     PG&E Corp. shares jumped $1.63 or 8 percent to...  paraphrase  
5     With 

<ipython-input-7-476c7c2df818>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
Skipping line 102: expected 5 fields, saw 6
Skipping line 656: expected 5 fields, saw 6
Skipping line 867: expected 5 fields, saw 6
Skipping line 880: expected 5 fields, saw 6
Skipping line 980: expected 5 fields, saw 6
Skipping line 1439: expected 5 fields, saw 6
Skipping line 1473: expected 5 fields, saw 6
Skipping line 1822: expected 5 fields, saw 6
Skipping line 1952: expected 5 fields, saw 6
Skipping line 2009: expected 5 fields, saw 6
Skipping line 2230: expected 5 fields, saw 6
Skipping line 2506: expected 5 fields, saw 6
Skipping line 2523: expected 5 fields, saw 6
Skipping line 2809: expected 5 fields, saw 6
Skipping line 2887: expected 5 fields, saw 6
Skipping line 2920: expected 5 fields, saw 6
Skipping line 2944: expected 5 fields, saw 6
Skip

## Quora Dataset

In [24]:
df = load_data(
    "quora_duplicate_questions.tsv", "question1", "question2", "is_duplicate"
)
q_train, q_test = train_test_split(df)
print(q_train)
print(q_test)

<ipython-input-7-476c7c2df818>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)


                                               input_text  \
332000               How do I get answers to my question?   
321555     How can I treat a dry throat and a runny nose?   
347530  What is the difference between the Communist P...   
92894   What are Hillary Clinton's plans regarding Put...   
266777  How can I increase my followers and likes on I...   
...                                                   ...   
172501                              How do I design apps?   
236855                    How could I improve my English?   
143086                   How do you have American accent?   
234561                 How can I learn how to pickpocket?   
212433  Am not starting big? How can I make $1000 per ...   

                                              target_text      prefix  
332000         How can I get more answers on my question?  paraphrase  
321555  What causes a dry throat and a runny nose? How...  paraphrase  
347530  Why did the Communist Party of India split 

## Parabak Dataset

In [26]:
paradata = pd.read_csv("parabank_5m.tsv", sep='\t', header=None, error_bad_lines=False)

Streaming output truncated to the last 5000 lines.
Skipping line 1387401: expected 2 fields, saw 3
Skipping line 1387414: expected 2 fields, saw 3
Skipping line 1388204: expected 2 fields, saw 3
Skipping line 1388693: expected 2 fields, saw 3
Skipping line 1389233: expected 2 fields, saw 3
Skipping line 1389248: expected 2 fields, saw 4
Skipping line 1389441: expected 2 fields, saw 3
Skipping line 1389636: expected 2 fields, saw 3
Skipping line 1390851: expected 2 fields, saw 3
Skipping line 1391404: expected 2 fields, saw 3
Skipping line 1392552: expected 2 fields, saw 3
Skipping line 1392865: expected 2 fields, saw 3
Skipping line 1393098: expected 2 fields, saw 3
Skipping line 1393413: expected 2 fields, saw 3
Skipping line 1394616: expected 2 fields, saw 3
Skipping line 1394964: expected 2 fields, saw 4
Skipping line 1395353: expected 2 fields, saw 3
Skipping line 1398921: expected 2 fields, saw 3
Skipping line 1399615: expected 2 fields, saw 3
Skipping line 1401546: expected 2 fie

In [29]:
paradata = paradata[0:2355090]

paradata['prefix'] = 'paraphrase'
paradata.rename(columns={0:'input_text',1:'target_text'}, inplace = True)
para_train, para_test = train_test_split(paradata)

print(para_train)
print(para_test)

                                                input_text  \
1113066                      So , you boys from Arkansas ?   
880079   Can the authorities pay part or all of the mai...   
749353                                         I 's sick !   
1743021         That statement was technically not a lie .   
1898845                                 One kick and ... !   
...                                                    ...   
1495136  How national rules take effect Laws are signed...   
2303746                           Yes , since you insist .   
1211781                   I tell you what you need to do .   
600817     Article 4(2)(e ) is replaced by the following :   
1331387   Here is the American Tenente ,  he said in It...   

                                               target_text      prefix  
1113066                    So you guys are from Arkansas ?  paraphrase  
880079   Can the authorities pay part or all of the mai...  paraphrase  
749353                              

### Data Transformation and Cleaning

In [28]:
train_df = pd.concat([train_df, q_train,para_train])
eval_df = pd.concat([eval_df, q_test,para_test])

train_df = train_df[["prefix", "input_text", "target_text"]]
eval_df = eval_df[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
eval_df = eval_df.dropna()

train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)



In [30]:
print(train_df)
print(eval_df)

             prefix                                         input_text  \
0        paraphrase  Amrozi accused his brother, whom he called "th...   
2        paraphrase  They had published an advertisement on the Int...   
4        paraphrase  The stock rose $2.11, or about 11 percent, to ...   
5        paraphrase  Revenue in the first quarter of the year dropp...   
7        paraphrase  The DVD-CCA then appealed to the state Supreme...   
...             ...                                                ...   
1417376  paraphrase         Yes, may I speak to Anne Brighton, please?   
890872   paraphrase            He walks, talks, interacts with people.   
189065   paraphrase                                          Antidote.   
469353   paraphrase  The first ground of appel Arguments of the par...   
1124907  paraphrase  By proposing to amend the Accounting Directive...   

                                               target_text  
0        Referring to him as only "the witness", A

## Data is ready for training